# CodeBERT Similarity Calculations
A consolidated notebook to run PSG, SG, and TPU-SG similarity scoring and persist results.

**Sections**
1. Configure Paths and Environment
2. Import Dependencies and Utilities
3. Load Reference File and Discover Candidates
4. Run Similarity Scoring Loop (All Three Scripts)
5. Append and Merge CSV Outputs
6. Quick Summary and Verification

In [ ]:
# 1. Configure Paths and Environment
from pathlib import Path
import sys

if "__file__" in globals():
    SCRIPT_DIR = Path(__file__).resolve().parent
else:
    # Notebook execution: assume cwd is the repo root
    SCRIPT_DIR = Path.cwd() / "codebertscore-similarity/analyser-results"
    if not SCRIPT_DIR.exists():
        # Fallback to cwd if running from inside analyser-results already
        SCRIPT_DIR = Path.cwd()

# Derive project root based on known layout
if SCRIPT_DIR.name == "analyser-results" and SCRIPT_DIR.parent.name == "codebertscore-similarity":
    PROJECT_ROOT = SCRIPT_DIR.parent.parent
else:
    PROJECT_ROOT = SCRIPT_DIR

CODE_BERT_PACKAGE = PROJECT_ROOT / "codebertscore-similarity"
sys.path.insert(0, str(CODE_BERT_PACKAGE))

# Per-script configuration matching the original files
CONFIGS = {
    "psg": {
        "reference": PROJECT_ROOT / "codebertscore-similarity/references/TFLite_detection_video.py",
        "candidate_glob": "*/exported_valid_code/psg/*.py",
        "output_csv": PROJECT_ROOT / "codebertscore-similarity/analyser-results/similarity_results_psg.csv",
        "lang": "python",
    },
    "sg": {
        "reference": PROJECT_ROOT / "codebertscore-similarity/references/object_color_classify.ino",
        "candidate_glob": "*/exported_valid_code/sg/*.ino",
        "output_csv": PROJECT_ROOT / "codebertscore-similarity/analyser-results/similarity_results_sg.csv",
        "lang": "c_sharp",
    },
    "tpusg": {
        "reference": PROJECT_ROOT / "codebertscore-similarity/references/TFLite_detection_video_TPU.py",
        "candidate_glob": "*/exported_valid_code/tpusg/*.py",
        "output_csv": PROJECT_ROOT / "codebertscore-similarity/analyser-results/similarity_results_tpusg.csv",
        "lang": "python",
    },
}

# Include both 2025 and 2026 runs under langfuse_export
CANDIDATE_ROOT = PROJECT_ROOT / "langfuse_export"

In [ ]:
# 2. Import Dependencies and Utilities
import os
from datetime import datetime
import pandas as pd
import torch
import code_bert_score

# Verify PyTorch version (must be 2.6+ for security fix CVE-2025-32434)
torch_version = torch.__version__.split('+')[0]  # Remove +cu124 suffix
torch_major, torch_minor = map(int, torch_version.split('.')[:2])
if torch_major < 2 or (torch_major == 2 and torch_minor < 6):
    print("╔═════════════════════════════════════════════════════════════════╗")
    print("║  ❌ ERROR: PyTorch version too old!                             ║")
    print("╚═════════════════════════════════════════════════════════════════╝")
    print(f"\nCurrent version: {torch.__version__}")
    print(f"Required version: ≥ 2.6.0")
    print()
    print("PyTorch was upgraded in the terminal, but this kernel is still using the old version.")
    print()
    print("🔧 FIX: Restart the kernel to load PyTorch 2.6.0")
    print("   1. Click 'Kernel' → 'Restart Kernel' (or press Ctrl+Shift+P)")
    print("   2. Re-run all cells from the beginning")
    print()
    raise RuntimeError(f"PyTorch {torch.__version__} is too old. Need 2.6.0+ for CVE-2025-32434 fix. Restart kernel!")

print(f"✓ PyTorch version: {torch.__version__}")

# Enable CUDA optimizations for better performance
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.set_grad_enabled(False)  # Disable gradients for inference-only
    
    # Get GPU info
    gpu_name = torch.cuda.get_device_name()
    compute_cap = torch.cuda.get_device_capability()
    total_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Compute Capability: {compute_cap[0]}.{compute_cap[1]}")
    print(f"✓ Memory: {total_memory_gb:.1f} GB")
    
    # Enable TF32 on Ampere+ GPUs (RTX 30xx, A100, etc.) - Pascal/GTX 1060 doesn't support this
    if compute_cap[0] >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("✓ Enabled TF32 precision (Ampere+ GPU)")
    else:
        print(f"ℹ TF32 not available (requires Ampere+ GPU, you have compute {compute_cap[0]}.{compute_cap[1]})")
    
    print("✓ CUDA optimizations enabled")


def load_file_content(file_path: Path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None


def extract_model_name(candidate_path: Path):
    # .../<run>/exported_valid_code/<variant>/file.ext
    try:
        return candidate_path.parents[2].name
    except IndexError:
        return candidate_path.stem


def ensure_header(csv_path: Path, columns):
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    if not csv_path.exists():
        pd.DataFrame(columns=columns).to_csv(csv_path, index=False)


def get_processed_ids(csv_path: Path):
    if not csv_path.exists():
        return set()
    try:
        df = pd.read_csv(csv_path, usecols=["Candidate_ID"])
        if "Candidate_ID" in df.columns:
            return set(df["Candidate_ID"].astype(str).dropna())
    except Exception as e:
        print(f"Error reading existing CSV {csv_path}: {e}")
    return set()


def get_processed_paths(csv_path: Path):
    if not csv_path.exists():
        return set()
    try:
        df = pd.read_csv(csv_path, usecols=["Candidate_Path"])
        if "Candidate_Path" in df.columns:
            return set(df["Candidate_Path"].astype(str).dropna())
    except Exception as e:
        print(f"Error reading existing CSV {csv_path}: {e}")
    return set()


def parse_model_fields(raw_model):
    """Extract base model (phi4|qw32|gpt5) and prompt_level (abla-l1|abla-l2|original) from raw model text."""
    text = str(raw_model)
    if "_" in text:
        _, text = text.split("_", 1)
        
    levels = ["abla-l1", "abla-l2", "abla-1p", "abla-2p"]
    prompt_level = next((lvl for lvl in levels if lvl in text), "original")
    base_model = next((m for m in ("phi4", "qw32", "gpt5") if m in text), text.split("-")[0])
    
    # Normalize model names
    model_mapping = {
        "gpt5": "gpt-5",
        "qw32": "qwen32",
    }
    base_model = model_mapping.get(base_model, base_model)
    
    return base_model, prompt_level

In [ ]:
# 3. Load Reference File and Discover Candidates
references = {}
candidate_sets = {}

for key, cfg in CONFIGS.items():
    ref_content = load_file_content(cfg["reference"])
    if not ref_content:
        print(f"Failed to load reference for {key}: {cfg['reference']}")
        continue
    references[key] = ref_content
    # search across all runs under langfuse_export (e.g., 2025, 2026, etc.)
    candidate_files = sorted(CANDIDATE_ROOT.glob(f"*/{cfg['candidate_glob']}"))
    candidate_sets[key] = candidate_files
    print(f"[{key}] reference loaded; candidates: {len(candidate_files)}")

# Device selection
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Batch size configuration based on GPU memory
if DEVICE == "cuda":
    total_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    compute_cap = torch.cuda.get_device_capability()
    
    # Auto-adjust batch size based on available VRAM
    if total_mem_gb <= 6:
        BATCH_SIZE = 20  # GTX 1060 6GB, GTX 1650, etc.
        print("⚠️  6GB GPU detected - using batch_size=20 to prevent OOM")
    elif total_mem_gb <= 8:
        BATCH_SIZE = 24  # 8GB cards
    elif total_mem_gb <= 10:
        BATCH_SIZE = 32  # RTX 3060 Ti, RTX 3080 (10GB)
    elif total_mem_gb <= 12:
        BATCH_SIZE = 48  # RTX 3060 (12GB)
    else:
        BATCH_SIZE = 64  # 16GB+ cards
    
    print(f"Using device: {DEVICE} ({total_mem_gb:.1f}GB)")
    print(f"Batch size: {BATCH_SIZE}")
else:
    BATCH_SIZE = 64
    print(f"Using device: {DEVICE}")

In [ ]:
# 4. Run Similarity Scoring Loop (All Three Scripts)
results = {}
common_columns = [
    "Candidate_ID",
    "Model",
    "Prompt_level",
    "Precision",
    "Recall",
    "F1",
    "F3",
    # "Reference_File",
    # "Language",
    "Processor",
    # "Timestamp",
]

for key, cfg in CONFIGS.items():
    ensure_header(cfg["output_csv"], common_columns)
    processed_ids = get_processed_ids(cfg["output_csv"])
    candidate_files = candidate_sets.get(key, [])
    ref_content = references.get(key)
    if not ref_content:
        continue

    loop_records = []
    for candidate_path in candidate_files:
        candidate_id = candidate_path.relative_to(PROJECT_ROOT).as_posix()
        if candidate_id in processed_ids:
            continue

        candidate_content = load_file_content(candidate_path)
        if not candidate_content:
            continue

        try:
            P, R, F1, F3 = code_bert_score.score(
                cands=[candidate_content],
                refs=[ref_content],
                lang=cfg["lang"],
                device=DEVICE,
                batch_size=BATCH_SIZE,
                verbose=False,
            )

            p_val = P[0].item()
            r_val = R[0].item()
            f1_val = F1[0].item()
            f3_val = F3[0].item()

            model_name = extract_model_name(candidate_path)
            base_model, prompt_level = parse_model_fields(model_name)
            timestamp = datetime.now().isoformat()

            record = {
                "Candidate_ID": candidate_id,
                "Model": base_model,
                "Prompt_level": prompt_level,
                "Precision": p_val,
                "Recall": r_val,
                "F1": f1_val,
                "F3": f3_val,
                # "Reference_File": cfg["reference"],
                # "Language": cfg["lang"],
                "Processor": key,
                # "Timestamp": timestamp,
            }
            loop_records.append(record)
        except Exception as e:
            print(f"[{key}] Error processing {candidate_path}: {e}")

    if loop_records:
        df_loop = pd.DataFrame(loop_records)
        df_loop.to_csv(cfg["output_csv"], mode="a", header=False, index=False)
        results[key] = df_loop
        print(f"[{key}] wrote {len(df_loop)} rows to {cfg['output_csv']}")
    else:
        print(f"[{key}] no new rows written")

In [ ]:
# 5. Append and Merge CSV Outputs
merged = []
run_candidate_counts = {k: len(v) for k, v in candidate_sets.items()}
output_row_counts = {}
for key, cfg in CONFIGS.items():
    if cfg["output_csv"].exists():
        try:
            df = pd.read_csv(cfg["output_csv"])
            df["run"] = key
            merged.append(df)
            output_row_counts[key] = len(df)
        except Exception as e:
            print(f"[{key}] Error reading {cfg['output_csv']}: {e}")

# 1. Filter out empty DataFrames to avoid the warning
valid_merged = [df for df in merged if not df.empty]

# 2. Check if there are any valid DataFrames left to concatenate
if valid_merged:
    combined = pd.concat(valid_merged, ignore_index=True)
    print(f"Combined rows: {len(combined)}")
else:
    combined = pd.DataFrame()
    print("No CSVs to combine yet.")

print("Run counts (candidates -> csv rows):")
for key in CONFIGS:
    cand = run_candidate_counts.get(key, 0)
    rows = output_row_counts.get(key, 0)
    print(f"{key}: {cand} candidates; {rows} rows in csv")
total_candidates = sum(run_candidate_counts.values())
total_rows = sum(output_row_counts.values())
print(f"Totals discovered: {total_candidates} candidates across runs; {total_rows} rows across csvs")

In [ ]:
# 6. Quick Summary and Verification
if not combined.empty:
    display(combined.head())
    display(combined.tail())
    print("Counts by run:")
    print(combined.groupby("run")["Candidate_ID"].count())
    print("F1 stats:")
    print(combined["F1"].describe())
    discovered = {k: len(v) for k, v in candidate_sets.items()}
    print("Coverage (rows vs discovered candidates):")
    for key in CONFIGS:
        rows = len(combined[combined["run"] == key])
        print(f"{key}: {rows} rows vs {discovered.get(key, 0)} candidates")
    print(f"Total candidates: {sum(discovered.values())}; total rows: {len(combined)}")
else:
    print("No data to summarize yet.")

In [ ]:
# 7. Backfill prompt_level and normalize Model for existing CSV outputs

def parse_from_candidate_id(candidate_id: str):
    """Derive base model and prompt level from the Candidate_ID path token."""
    text = str(candidate_id)
    run_segment = next(
        (seg for seg in Path(text).parts if "abla" in seg or "original" in seg or "og" in seg),
        text,
    )
    return parse_model_fields(run_segment)

def backfill_outputs():
    for c in ["psg", "sg", "tpusg"]:
        csv_path = CONFIGS[c]["output_csv"]
        if not csv_path.exists():
            print(f"{c}: {csv_path} missing, skipping")
            continue
        df = pd.read_csv(csv_path)
        if df.empty:
            print(f"{c}: {csv_path} empty, skipping")
            continue
        if "Candidate_ID" not in df.columns:
            print(f"{c}: Candidate_ID missing in {csv_path}, skipping backfill")
            continue
        base_prompt = df["Candidate_ID"].apply(parse_from_candidate_id).apply(pd.Series)
        df["Model"] = base_prompt[0]
        df["Prompt_level"] = base_prompt[1]
        if "Candidate_Path" in df.columns:
            df = df.drop(columns=["Candidate_Path"])
        ordered_cols = [
            "Candidate_ID",
            "Model",
            "Precision",
            "Recall",
            "F1",
            "F3",
            "Processor",
            "Prompt_level",
        ]
        df = df[[col for col in ordered_cols if col in df.columns]]
        df.to_csv(csv_path, index=False)
        print(f"{c}: cleaned {len(df)} rows -> {csv_path}")
    print("Backfill complete.")

backfill_outputs()
